# Building LLM Applications With PaLM API and LangChain
### Retrieval Augmented Generation (RAG) Using LangChain and PaLM API


In [1]:
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [2]:
import vertexai

PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

In [3]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
# Utils
import time
from typing import List

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})

/Users/derrickmwiti/mukimaenv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [5]:
loader = PyPDFLoader("activations.pdf")

In [6]:
documents = loader.load()

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

In [8]:
texts = text_splitter.split_documents(documents)

In [9]:
len(texts)

18

In [10]:
db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()

In [11]:
llm = ChatVertexAI()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [13]:
query = "What does the book talk about?"

In [14]:
answer = qa_stuff.run(query)
answer



> Entering new RetrievalQA chain...

> Finished chain.


' The book talks about activation functions in JAX and Flax.\n'

## Retrieval Augmented Generation (RAG) Using LangChain


In [28]:
qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [29]:
response = qa_with_sources_chain({"query":"What is the purpose of activation functions?"})

In [30]:
response

{'query': 'What is the purpose of activation functions?',
 'result': ' Activation functions are applied in neural networks to ensure that the network outputs the desired result. ',
 'source_documents': [Document(page_content='\x00\x00\x00 Final thoughts\nActivation functions are applied in neural networks to ensure that the network\noutputs the desired result. The activations functions cap the output within a\nspecific range. For instance, when solving a binary classification problem, the\noutcome should be a number between 0 and 1. This indicates the probability of\nan item belonging to either of the two classes. However, in a regression problem,\nyou want the numerical prediction of a quantity, for example, the price of an', metadata={'page': 0, 'source': 'activations.pdf'}),
  Document(page_content='07/07/2023, 09:18 Activation functions in JAX and Flax\nhttps://www .machinelearningnuggets.com/jax-ﬂax-activation-functions/ 2/14\x00\x00 ELU \x00 Exponential linear unit activation\n\x

In [31]:
print(response['result'])

 Activation functions are applied in neural networks to ensure that the network outputs the desired result. 


In [32]:
print(response['source_documents'])

[Document(page_content='\x00\x00\x00 Final thoughts\nActivation functions are applied in neural networks to ensure that the network\noutputs the desired result. The activations functions cap the output within a\nspecific range. For instance, when solving a binary classification problem, the\noutcome should be a number between 0 and 1. This indicates the probability of\nan item belonging to either of the two classes. However, in a regression problem,\nyou want the numerical prediction of a quantity, for example, the price of an', metadata={'page': 0, 'source': 'activations.pdf'}), Document(page_content='07/07/2023, 09:18 Activation functions in JAX and Flax\nhttps://www .machinelearningnuggets.com/jax-ﬂax-activation-functions/ 2/14\x00\x00 ELU \x00 Exponential linear unit activation\n\x00\x00 CELU \x00 Continuously-differentiable exponential linear unit\n\x00\x00 GELU\x00 Gaussian error linear unit activation\n\x00\x00\x00 GLU \x00 Gated linear unit activation\n\x00\x00\x00 Soft sign\n\